In [1]:
# Importing packages
import os
import yaml
import logging
import pandas as pd
from qiime2 import Artifact
from qiime2 import Visualization
from qiime2 import Metadata
import qiime2.plugins.dada2.actions as dada2_actions
import qiime2.plugins.metadata.actions as metadata_actions
from qiime2.plugins.feature_table.visualizers import tabulate_seqs
from qiime2.plugins.feature_table.visualizers import summarize
from qiime2.plugins.feature_table.visualizers import core_features

from qiime2.plugins.feature_table.methods import merge
from qiime2.plugins.feature_table.methods import merge_seqs
from qiime2.plugins.feature_table.methods import merge_taxa

from qiime2.plugins.feature_table.methods import filter_samples
from qiime2.plugins.feature_table.methods import filter_seqs

import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
experiment_name = "thayane-PM-joined"
base_dir = "/mnt/nupeb/rede-micro/redemicro-thayane"
interim_dir = f"{base_dir}/data/interim"

metadata_path = f'{base_dir}/data/metadata-meno-joined-hist.tsv'

single_end_path = f"{base_dir}/experiments/thayane-PM-single-end-trim/qiime-artifacts/"
paired_end_path = f"{base_dir}/experiments/thayane-meno-joined/qiime-artifacts/bkp-paired-meno"

In [4]:
metadata_artifact = Metadata.load(metadata_path)
metadata_artifact.to_dataframe()

,age,class
sample-id,,
M05,56.0,3
M33,52.0,3
M34,58.0,3
M35,58.0,3
M36,46.0,1
...,...,...
M03,59.0,3
M06,62.0,3
M09,53.0,3


In [5]:
single_tabs = Artifact.load(f"{single_end_path}/dada2-tabs.qza")
single_reps = Artifact.load(f"{single_end_path}/dada2-reps.qza")

paired_tabs = Artifact.load(f"{paired_end_path}/dada2-tabs.qza")
paired_reps = Artifact.load(f"{paired_end_path}/dada2-reps.qza")

In [6]:
# Merge feature tables
merged_feature_table = merge(
    tables = [single_tabs, paired_tabs],
).merged_table
merged_feature_table.view(pd.DataFrame).shape

(82, 2963)

In [7]:
# Merge sequences
merged_sequences = merge_seqs(
    data = [single_reps, paired_reps]
).merged_data
merged_sequences.view(pd.Series).shape

/home/lauro/anaconda3/envs/qiime2-2023.5/lib/python3.8/site-packages/q2_types/feature_data/_transformer.py:258: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for id_, seq in data.iteritems():


(2963,)

In [9]:
merged_feature_table.view(pd.DataFrame).index

Index(['M01', 'M03', 'M06', 'M09', 'M12', 'M19', 'M05', 'M07', 'M100', 'M102',
       'M103', 'M105', 'M106', 'M107', 'M109', 'M111', 'M112', 'M113', 'M116',
       'M117', 'M120', 'M121', 'M122', 'M124', 'M125', 'M126', 'M129', 'M130',
       'M133', 'M134', 'M135', 'M138', 'M140', 'M141', 'M143', 'M144', 'M148',
       'M150', 'M151', 'M155', 'M156', 'M160', 'M162', 'M164', 'M33', 'M34',
       'M35', 'M36', 'M37', 'M38', 'M39', 'M40', 'M44', 'M45', 'M46', 'M47',
       'M49', 'M52', 'M55', 'M56', 'M57', 'M58', 'M60', 'M64', 'M67', 'M69',
       'M72', 'M73', 'M75', 'M76', 'M77', 'M78', 'M79', 'M80', 'M86', 'M88',
       'M91', 'M93', 'M95', 'M97', 'M98', 'M99'],
      dtype='object')

In [10]:
# Create view and save frequency table
merged_feature_table.save(os.path.join(interim_dir, 'joined-meno-tabs.qza'))
tabs_view = summarize(table=merged_feature_table, sample_metadata=metadata_artifact).visualization
tabs_view.save(os.path.join(interim_dir, 'joined-meno-tabs.qzv'))

'/mnt/nupeb/rede-micro/redemicro-thayane/data/interim/joined-meno-tabs.qzv'

In [11]:
# Create view and save representative sequences
merged_sequences.save(os.path.join(interim_dir, 'joined-meno-reps.qza'))
reps_view = tabulate_seqs(data=merged_sequences).visualization
reps_view.save(os.path.join(interim_dir, 'joined-meno-reps.qzv'))

'/mnt/nupeb/rede-micro/redemicro-thayane/data/interim/joined-meno-reps.qzv'

In [9]:
qiime_path = f"{base_dir}/experiments/thayane-meno-joined/qiime-artifacts"

merged_feature_table.save(os.path.join(qiime_path, 'dada2-tabs.qza'))
tabs_view.save(os.path.join(qiime_path, 'dada2-tabs.qzv'))

merged_sequences.save(os.path.join(qiime_path, 'dada2-reps.qza'))
reps_view.save(os.path.join(qiime_path, 'dada2-reps.qzv'))

'/mnt/nupeb/rede-micro/redemicro-thayane/experiments/thayane-meno-joined/qiime-artifacts/dada2-reps.qzv'

In [15]:
!ls -lah {qiime_path}

total 8.2M
drwxrwxr-x+ 6 lauro redeminas 4.0K Feb 23 20:02 .
drwxrwxr-x+ 6 lauro redeminas 4.0K Feb 23 18:21 ..
drwxrwxr-x+ 2 lauro lauro     4.0K Feb 23 18:33 alpha-analysis
-rw-rw-r--+ 1 lauro lauro     352K Feb 23 18:20 alpha-rarefaction.qzv
-rw-rw-r--+ 1 lauro lauro     722K Feb 23 20:02 ancom-clr-f_statistic.qzv
-rw-rw-r--+ 1 lauro lauro     697K Feb 23 19:33 ancom-log-f_statistic.qzv
-rw-rw-r--+ 1 lauro lauro     697K Feb 23 19:04 ancom-sqrt-f_statistic.qzv
drwxrwxr-x+ 2 lauro lauro     4.0K Feb 23 18:35 beta-analysis
-rw-rw-r--+ 1 lauro lauro     854K Feb 23 18:20 beta-rarefaction-braycurtis-nj-spearman-564.qzv
drwxrwxr-x+ 2 lauro lauro     4.0K Feb 23 18:05 bkp-paired-meno
drwxrwxr-x+ 2 lauro lauro     4.0K Feb 23 18:06 bkp-single-meno
-rw-rw-r--+ 1 lauro lauro     289K Feb 23 18:19 dada2-reps.qza
-rw-rw-r--+ 1 lauro lauro     789K Feb 23 18:19 dada2-reps.qzv
-rw-rw-r--+ 1 lauro lauro     210K Feb 23 18:19 dada2-tabs.qza
-rw-rw-r--+ 1 lauro lauro     577K Feb 23 18:19 dada2-tab